In [1]:
from dotenv import load_dotenv, find_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

# Load environment variables
load_dotenv(find_dotenv())

True

In [7]:
# Load the web page
page = WebBaseLoader("https://www.wikihow.com/Do-Yoga")
web_page_document = page.load()

#print("Loaded documents:", len(web_page_document))

# Split documents
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(documents=web_page_document)

# Initialize components
llm = ChatOpenAI()
embeddings = OpenAIEmbeddings()
vector_db = Chroma.from_documents(documents, embeddings)

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import MessagesPlaceholder

retriever=vector_db.as_retriever()

chat_prompt=ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user","{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to conversation")
    ]

)

In [11]:
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain_core.messages import AIMessage
from langchain_core.messages.human import HumanMessage
retriever_chain=create_history_aware_retriever(llm,retriever,chat_prompt)
sample_answer="""
                Some points for Yoga beginner are :
                Silence your phone, turn off screens, and let others in your household know you're not to be disturbed.
                The more you practice, the easier it'll be to devote time to yoga every day.
                Try scheduling your at-home practices at the beginning of the week, marking in your calendar how long each flow is and when you’ll complete it.
              """  
chat_history=[
                HumanMessage(content="What are the things if someone consider to start yoga"),
                AIMessage(content=sample_answer)
             ]


In [14]:
chat_prompt=ChatPromptTemplate.from_messages( [
    ("system","Answer the user's questions based on the below context:\n\n {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}")
  ]
)

document_chain=create_stuff_documents_chain(llm,chat_prompt)
retrieval_chain=create_retrieval_chain(retriever_chain,document_chain)

In [19]:
output=retrieval_chain.invoke({"chat_history":chat_history, "input":"How do i start yoga"})
print(output["answer"])

To start yoga, you can begin by finding a comfortable place to practice, devoting a set amount of time to your practice, setting a goal for your practice, choosing a yoga style that works best for you, and following a guided video to begin your at-home yoga practice. You can also find free beginner yoga practices on platforms like YouTube or use yoga apps like Down Dog, Daily Yoga, or Gaia for instructional material. Practicing in a way that is enjoyable for you is key to sticking with it, so adapt your practice and poses to suit your body and preferences.
